In [2]:
import pickle
embeddings = {}
with open('data/commonvoice_embeddings.pkl', 'rb') as file:
    embeddings = pickle.load(file)
import pandas as pd
df = pd.read_csv("./data/cv-corpus-21.0-delta-2025-03-14/en/validated.tsv", delimiter="\t")
value_dict = df[["path", "age"]]
value_dict = value_dict.set_index("path").rename_axis(None)
value_dict = value_dict.to_dict()["age"]
value_dict

{'common_voice_en_41923025.mp3': 'thirties',
 'common_voice_en_42356358.mp3': 'teens',
 'common_voice_en_42165090.mp3': nan,
 'common_voice_en_41921729.mp3': 'nineties',
 'common_voice_en_42528393.mp3': 'teens',
 'common_voice_en_42006138.mp3': nan,
 'common_voice_en_42555516.mp3': 'twenties',
 'common_voice_en_41951792.mp3': 'thirties',
 'common_voice_en_42446785.mp3': 'fifties',
 'common_voice_en_41974998.mp3': 'twenties',
 'common_voice_en_42020013.mp3': 'twenties',
 'common_voice_en_42046273.mp3': 'twenties',
 'common_voice_en_42458975.mp3': 'fourties',
 'common_voice_en_42494012.mp3': 'thirties',
 'common_voice_en_42435518.mp3': 'twenties',
 'common_voice_en_41930868.mp3': nan,
 'common_voice_en_42251480.mp3': 'sixties',
 'common_voice_en_41923270.mp3': 'thirties',
 'common_voice_en_42594358.mp3': 'thirties',
 'common_voice_en_42141720.mp3': nan,
 'common_voice_en_42498572.mp3': 'twenties',
 'common_voice_en_42216083.mp3': 'fourties',
 'common_voice_en_42466107.mp3': 'teens',
 'co

In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import numpy as np
X = []
true_label = []
cluster_names = []
for key, val in embeddings.items():
    tl = value_dict[f"{key}.mp3"]
    if tl in ["twenties", "thirties", "sixties", "teens"]:
        X.append(val)
        true_label.append(tl)
        if tl not in cluster_names:
            cluster_names.append(tl)
X = np.array(X)

kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(X)
cluster_labels = kmeans.labels_
print(classification_report(true_label, [cluster_names[l] for l in cluster_labels]))


              precision    recall  f1-score   support

     sixties       0.04      0.13      0.07        31
       teens       0.00      0.00      0.00        14
    thirties       0.38      0.61      0.47        56
    twenties       0.81      0.30      0.44       126

    accuracy                           0.33       227
   macro avg       0.31      0.26      0.24       227
weighted avg       0.55      0.33      0.37       227



/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rafkiep1/Desktop/AI_ethics_final/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [4]:
import numpy as np
confusion_matrix = np.zeros((4, 4))
buckets = ["twenties", "thirties", "sixties", "teens"]

for idx, cl in enumerate(cluster_labels):
    confusion_matrix[buckets.index(true_label[idx])][cl] += 1
print(confusion_matrix)
purity_score = np.sum([np.max(r) for r in confusion_matrix])/len(cluster_labels)
print(purity_score)

[[38. 25. 63.  0.]
 [ 5. 34. 17.  0.]
 [ 1. 26.  4.  0.]
 [ 3.  4.  7.  0.]]
0.5726872246696035
